In [2]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

Given a location and a maximum distance (in meters), the query should return the restaurants within the given max distance.<br>
The input location is expressed in terms of **latitute** and **longitude**.<br>
This is the basic query: many of the following ones will be extensions of this one to show the variety of filters that may be applied in the restaurant selection.

In [3]:
# My coordinates (latitude and longitude) 
# to be given as input location
my_latitude = 21.924238
my_longitude = -159.510035

# Distance in [meters] within which I am filtering
radius = 1500  

# Query to find restaurants near you
restaurant_close_rating = meta.aggregate([
    {
        "$geoNear": {
            # My location
            "near": {"type": "Point", "coordinates": [
                my_longitude, my_latitude]},  
            # Field for distance calculation
            "distanceField": "distance",
            # Max distance in meters  
            "maxDistance": radius,
            # Use spherical geometry for accurate distances  
            "spherical": True  
        }
    },
    {   # Create a temporary field for distance in meters
        "$addFields": {  
            "distance_meters": {"$round": ["$distance", 2]}  
        }
    },
    {
        "$project": {"_id":1,
                     "name":1,
                     "distance_meters":1}
    }
])

# Convert to list and print
results = list(restaurant_close_rating)
for business in results:
    print(business)

{'_id': ObjectId('67c751ad8c4ae1c3ac1be454'), 'name': 'Kauai Poke Co.Food Truck', 'distance_meters': 273.82}
{'_id': ObjectId('67c751b08c4ae1c3ac1c22cf'), 'name': 'The Fresh Shave', 'distance_meters': 277.07}
{'_id': ObjectId('67c751ad8c4ae1c3ac1bec13'), 'name': 'Uncle Bobby’s', 'distance_meters': 288.85}
{'_id': ObjectId('67c751b08c4ae1c3ac1c1eb5'), 'name': 'Kalaheo Cafe & Coffee Company', 'distance_meters': 1421.62}
{'_id': ObjectId('67c751af8c4ae1c3ac1c1853'), 'name': 'Brick Oven Pizza', 'distance_meters': 1439.32}
